In [ ]:
import pandas as pd

df = pd.read_csv("C:/Users/lgino/code/Lothardill/Hospitalis/Teo_final_gambit.csv")

# Step 1: Filter dataset to include only products where "Prix HT" >= 50€
df_filtered = df[df["Prix HT"] >= 50]

# Step 2: Precompute the Depot_1_Percentage once (before the loop)
df_depot_percentage = (
    df_filtered.groupby("Réf. Article")["Dépôt DMI"]
    .agg(lambda x: (x.sum() / x.count()) * 100)  # Calculate percentage
    .reset_index()
    .rename(columns={"Dépôt DMI": "Depot_1_Percentage"})
)

# Merge to bring "Depot_1_Percentage" into df_filtered
df_filtered = df_filtered.merge(df_depot_percentage, on="Réf. Article", how="left")

# Step 3: Initialize an empty dictionary
product_dict = {}

# Step 4: Loop through the dataframe
for _, row in df_filtered.iterrows():
    if row["Prix HT"] > 50 and row["Depot_1_Percentage"] >= 75:
        product_dict[row["Réf. Article"]] = row.to_dict()  # Store row as a dictionary


pd.DataFrame.from_dict(product_dict)

# Step 5: Display final results
print("✅ Dictionary stored with", len(product_dict), "products.")

C:\Users\lgino\AppData\Local\Temp\ipykernel_26104\2866567216.py:3: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/lgino/code/Lothardill/Hospitalis/Teo_final_gambit.csv")


In [3]:
import pandas as pd
df = pd.read_csv("C:/Users/lgino/code/Lothardill/Hospitalis/Teo_final_gambit.csv")
df.info()

C:\Users\lgino\AppData\Local\Temp\ipykernel_34892\3498937880.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/lgino/code/Lothardill/Hospitalis/Teo_final_gambit.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5953297 entries, 0 to 5953296
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   Fournisseur      object 
 2   Routage          object 
 3   Robot ETS        object 
 4   Date Insertion   object 
 5   Commande         object 
 6   Dépôt DMI        int64  
 7   Réf. Article     object 
 8   Libellé Article  object 
 9   Quantité         int64  
 10  Prix HT          float64
 11  Prix TTC         float64
 12  Mnt HT           float64
 13  Mnt TTC          float64
dtypes: float64(4), int64(3), object(7)
memory usage: 635.9+ MB


In [5]:
df["Date Insertion"] = pd.to_datetime(df["Date Insertion"], errors='coerce')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5953297 entries, 0 to 5953296
Data columns (total 14 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Unnamed: 0       int64         
 1   Fournisseur      object        
 2   Routage          object        
 3   Robot ETS        object        
 4   Date Insertion   datetime64[ns]
 5   Commande         object        
 6   Dépôt DMI        int64         
 7   Réf. Article     object        
 8   Libellé Article  object        
 9   Quantité         int64         
 10  Prix HT          float64       
 11  Prix TTC         float64       
 12  Mnt HT           float64       
 13  Mnt TTC          float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(6)
memory usage: 635.9+ MB


In [9]:
dict_ref_article_unique={}
dict_ref_article_multiple={}
for row in df["Réf. Article"]:
    if df["Réf. Article"].unique:
        dict_ref_article_unique.append(row, ignore_index=True)
        print(dict_ref_article_unique)

AttributeError: 'dict' object has no attribute 'append'

In [15]:
df["Réf. Article"] = df["Réf. Article"].astype(str).fillna("UNKNOWN")

In [19]:
import pandas as pd

# Step 2: Count occurrences of each "Réf. Article"
article_counts = df["Réf. Article"].value_counts()

# Step 3: Initialize empty DataFrames
df_unique = pd.DataFrame(columns=df.columns)
df_multiple = pd.DataFrame(columns=df.columns)

# Step 4: Loop through dataframe and classify into correct DataFrame
for _, row in df.iterrows():
    ref_article = row["Réf. Article"]

    if article_counts.get(ref_article, 0) == 1:  # Unique products
        df_unique = pd.concat([df_unique, row.to_frame().T], ignore_index=True)
    else:  # Products that appear multiple times
        df_multiple = pd.concat([df_multiple, row.to_frame().T], ignore_index=True)

# Step 5: Display results
print(f"✅ Unique Articles: {df_unique.shape[0]}")
print(f"✅ Multiple Occurrence Articles: {df_multiple.shape[0]}")

KeyboardInterrupt: 

In [13]:
df["Réf. Article"].value_counts()

Réf. Article
XXXXXXX      24278
0            21895
12106450S    16960
18060085S    14344
18965035S    13778
             ...  
VLT4047C         1
DFX9006SC        1
002022420        1
ZVL14030         1
141.2520         1
Name: count, Length: 129211, dtype: int64

In [22]:
import pandas as pd

# Step 1: Ensure "Réf. Article" has no NaN values
df["Réf. Article"] = df["Réf. Article"].astype(str).fillna("UNKNOWN")

# Step 2: Count occurrences of each "Réf. Article"
article_counts = df["Réf. Article"].value_counts()

# Step 3: Classify products based on their count (Vectorized)
df["Is_Unique"] = df["Réf. Article"].map(article_counts) == 1

# Step 4: Create separate DataFrames for unique & multiple occurrence products
df_unique = df[df["Is_Unique"]].drop(columns=["Is_Unique"])
df_multiple = df[~df["Is_Unique"]].drop(columns=["Is_Unique"])

# Step 5: Save the results
df_unique.to_csv("unique_articles.csv", index=False, encoding="utf-8-sig")
df_multiple.to_csv("multiple_articles.csv", index=False, encoding="utf-8-sig")

print(f"✅ Processing completed! Unique Articles: {df_unique.shape[0]}, Multiple Articles: {df_multiple.shape[0]}")

✅ Processing completed! Unique Articles: 33504, Multiple Articles: 5919793


In [32]:
df_multiple = df_multiple.drop(columns=["Unnamed: 0"])

KeyError: "['Unnamed: 0'] not found in axis"

In [36]:
df_multiple["Prix HT"] = df_multiple["Prix HT"].fillna("0.0")

In [88]:
df_multiple["Réf. Article"].value_counts().head(50)

Réf. Article
12106450S        16960
18060085S        14344
18965035S        13778
377179           10770
LNQ11            10658
31251180S        10562
18965040S        10273
377177            9701
W2DR01            7661
18965045S         7510
8371CEB           7127
TE1626W           6756
C13               6699
422627            6616
DKB7127           6482
3400891068238     6399
3400891343281     6382
300865            6307
335330            6053
AR-1588RT-2J      6011
CMRM6122INT       5943
72202468          5933
C0R47             5561
508538            5413
18965050S         4959
657314            4924
003-40F           4741
AR-7500-1         4734
210190            4644
422624            4641
4722              4630
30600090S         4507
EGIA60AMT         4464
037102            4389
8886803712        4388
30600095S         4348
383069            4346
220300            4314
657316            4282
332800-000010     4256
309110            4247
300629            4246
201971            423

In [82]:
df_multiple = df_multiple[df_multiple["Réf. Article"] != "XXXXXX"]
df_multiple = df_multiple[df_multiple["Réf. Article"] != "0"]
df_multiple = df_multiple[df_multiple["Réf. Article"] != "nan"]
df_multiple = df_multiple[df_multiple["Réf. Article"] != "."]
df_multiple = df_multiple[df_multiple["Réf. Article"] != "???"]

In [83]:
df_multiple.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5853200 entries, 0 to 5953296
Data columns (total 13 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Fournisseur      object        
 1   Routage          object        
 2   Robot ETS        object        
 3   Date Insertion   datetime64[ns]
 4   Commande         object        
 5   Dépôt DMI        int64         
 6   Réf. Article     object        
 7   Libellé Article  object        
 8   Quantité         int64         
 9   Prix HT          object        
 10  Prix TTC         float64       
 11  Mnt HT           float64       
 12  Mnt TTC          float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(7)
memory usage: 625.2+ MB


In [114]:
df_numeric_only = df_multiple[df_multiple["Réf. Article"].str.match(r"^\d+$", na=False)]
df_numeric_only["Réf. Article"].count()


2342210

In [108]:
df_multiple["Length"] = df_multiple["Réf. Article"].astype(str).str.len()
df_outliers_length['Réf. Article'].count()


52593

In [110]:
df_special_chars = df_multiple[~df_multiple["Réf. Article"].str.match(r"^[A-Za-z0-9\-]+$", na=False)]
df_special_chars["Réf. Article"].count()

264434

In [128]:
len(df_multiple["Réf. Article"].unique())

95702